<h1>Section 3.2.3 Coal Distribution with Transshipment</h1>

In this notebook we use <code>pulp</code> to solve the coal distribuion with transshipment problem from the text.

In [0]:
!pip install pulp

import numpy as np
import pulp

In [0]:
CostTo=np.array([
    [4,3],
    [5,4],
    [2,4]
])
CostFrom=np.array([
    [3,5,4,3],
    [4,3,4,4]
])
print(CostTo)
print(CostFrom)

We first create LpProblem in pulp.

In [0]:
Coal2LP = pulp.LpProblem("Coal Transshipment Problem",pulp.LpMinimize)

We next create the decision variables.  These come in two types: $x_{i,j}$ is the amount shipped from mine $i$ to station $j$, while $y_{j,k}$ is the amount shipped from station $j$ to city $k$.

In [0]:
Mines = range(3)
Stations = range(2)
Cities = range(4)

x = pulp.LpVariable.dicts("x",(Mines,Stations),0,None,pulp.LpContinuous)
y = pulp.LpVariable.dicts("y",(Stations,Cities),0,None,pulp.LpContinuous)

Now we add the objective function.

In [0]:
Coal2LP += sum(x[mine][station]*CostTo[mine,station] for mine in Mines for station in Stations) + sum(y[station][city]*CostFrom[station,city] for station in Stations for city in Cities ), "Total Cost"

In [0]:
Coal2LP

We need to add three types of contraints.
1. Amount shipped to each city does not exceed demand.
2. Amount shipped from each mine does not exceed supply.
3. Amounts shipped in and out of each station are equal.

In [0]:
Demand=[4,3,5,2]
for city in Cities:
    Coal2LP += sum(y[station][city] for station in Stations) >= Demand[city]
    
Supply=[5,5,4]
for mine in Mines:
    Coal2LP += sum(x[mine][station] for station in Stations) <= Supply[mine]
    
for station in Stations:
    Coal2LP += sum(x[mine][station] for mine in Mines) == sum(y[station][city] for city in Cities)

In [0]:
Coal2LP

Our LP is complete and we are ready to solve.

In [0]:
Coal2LP.solve()
print("Status:", pulp.LpStatus[Coal2LP.status])

In [0]:
pulp.value(Coal2LP.objective)

In [0]:
for v in Coal2LP.variables():
    print(v.name, "=", v.varValue)

Finally we can print the solution as a pair of arrays.

In [0]:
SolutionArray=np.array([[x[mine][station].varValue for station in Stations] for mine in Mines])
SolutionArray2=np.asarray([[y[station][city].varValue for city in Cities] for station in Stations])

print(SolutionArray)
print(SolutionArray2)